In [1]:
import os
os.getcwd() 

'/Users/biancaziliotto/Desktop/GitRepo/MLx2'

In [2]:
os.chdir('src')

In [3]:
from MLP import MLP 

In [4]:
mlp = MLP([3, 4, 2], 2, 1, "id")


In [5]:
mlp.layers
mlp.input_size
mlp.output_size

1

In [6]:
import numpy as np
X = np.ndarray((4,2))
X[0, :]= [1, 1]
X[1, :]= [1, 2]
X[2, :]= [1, 3]
X[3, :]= [1, 4]
y_true  = np.ndarray((4,1))
y_true[:,0] = [2,3,4,5]

print(X)
print(y_true)
z = np.concatenate((X,y_true), axis = 1)
print(type(z))
np.random.shuffle(z)
prova= np.array_split(z, 2, axis = 1)
print(prova[0])

print(X.sum(axis = 0, keepdims=True))


[[1. 1.]
 [1. 2.]
 [1. 3.]
 [1. 4.]]
[[2.]
 [3.]
 [4.]
 [5.]]
<class 'numpy.ndarray'>
[[1. 4.]
 [1. 1.]
 [1. 3.]
 [1. 2.]]
[[ 4. 10.]]


In [10]:
#mlp.fit(X, y_true, n_epochs = 500, batch_size = 2, error = "MSE", regularization = "no", weights_initialization = "scaled", weights_scale = 0.1, step = 0.06, momentum = 0.03)
mlp.fit(X, y_true, n_epochs = 500, batch_size = 2, step = 0.05, momentum = 0.05, regularization = "elastic", alpha_l1 = 0.01, alpha_l2 = 0.01)

Epoch 0: MSE = 9.119443013819708
Epoch 1: MSE = 6.1780302446120325
Epoch 2: MSE = 4.276032954756721
Epoch 3: MSE = 2.9941017976220072
Epoch 4: MSE = 2.1517155463461126
Epoch 5: MSE = 1.6682663273522136
Epoch 6: MSE = 1.4320273114193325
Epoch 7: MSE = 1.3046326751533055
Epoch 8: MSE = 1.263011579002059
Epoch 9: MSE = 1.2410222179803303
Epoch 10: MSE = 1.2445302052605811
Epoch 11: MSE = 1.2324439148168764
Epoch 12: MSE = 1.2236769828476066
Epoch 13: MSE = 1.229418446376073
Epoch 14: MSE = 1.2182738100737698
Epoch 15: MSE = 1.2138034049787432
Epoch 16: MSE = 1.2073931301216632
Epoch 17: MSE = 1.2035606666806344
Epoch 18: MSE = 1.2017025073824574
Epoch 19: MSE = 1.1930964036417109
Epoch 20: MSE = 1.1812874096021784
Epoch 21: MSE = 1.181392571800833
Epoch 22: MSE = 1.1651412021560663
Epoch 23: MSE = 1.1483929074170973
Epoch 24: MSE = 1.1451803006137915
Epoch 25: MSE = 1.1224966026256487
Epoch 26: MSE = 1.0963858132593
Epoch 27: MSE = 1.0738557312769434
Epoch 28: MSE = 1.0444767673845803
Epo

In [8]:

mlp.predict(X)

array([[2.08073198],
       [3.03646102],
       [3.99219005],
       [4.94791908]])